In [2]:
!pip install folium

  Using cached folium-0.17.0-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached branca-0.7.2-py3-none-any.whl.metadata (1.5 kB)
Using cached folium-0.17.0-py2.py3-none-any.whl (108 kB)
Using cached branca-0.7.2-py3-none-any.whl (25 kB)


In [6]:
import folium
import pandas as pd
import csv

In [9]:
import zipfile

In [14]:
# Define the extraction directory and the zip file path
extraction_directory = 'fordbike'
zip_file_path = '201902-fordgobike-tripdata.csv.zip'

# Extract the zip file to the specified directory
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_directory)

# Verify the extraction by listing the contents of the directory
import os
extracted_files = os.listdir(extraction_directory)
extracted_files

['201902-fordgobike-tripdata.csv']

In [16]:
df=pd.read_csv("fordbike/201902-fordgobike-tripdata.csv")
df.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip
0,52185,2019-02-28 17:32:10.1450,2019-03-01 08:01:55.9750,21.0,Montgomery St BART Station (Market St at 2nd St),37.789625,-122.400811,13.0,Commercial St at Montgomery St,37.794231,-122.402923,4902,Customer,1984.0,Male,No
1,42521,2019-02-28 18:53:21.7890,2019-03-01 06:42:03.0560,23.0,The Embarcadero at Steuart St,37.791464,-122.391034,81.0,Berry St at 4th St,37.775880,-122.393170,2535,Customer,NaN,NaN,No
2,61854,2019-02-28 12:13:13.2180,2019-03-01 05:24:08.1460,86.0,Market St at Dolores St,37.769305,-122.426826,3.0,Powell St BART Station (Market St at 4th St),37.786375,-122.404904,5905,Customer,1972.0,Male,No
3,36490,2019-02-28 17:54:26.0100,2019-03-01 04:02:36.8420,375.0,Grove St at Masonic Ave,37.774836,-122.446546,70.0,Central Ave at Fell St,37.773311,-122.444293,6638,Subscriber,1989.0,Other,No
4,1585,2019-02-28 23:54:18.5490,2019-03-01 00:20:44.0740,7.0,Frank H Ogawa Plaza,37.804562,-122.271738,222.0,10th Ave at E 15th St,37.792714,-122.248780,4898,Subscriber,1974.0,Male,Yes


In [17]:
# Extract the relevant columns: station name, latitude, longitude, and number of rides
station_data = df.groupby(['start_station_name', 'start_station_latitude', 'start_station_longitude']).size().reset_index(name='number_of_rides')
station_data.head()

,start_station_name,start_station_latitude,start_station_longitude,number_of_rides
0,10th Ave at E 15th St,37.792714,-122.248780,57
1,10th St at Fallon St,37.797673,-122.262997,353
2,10th St at University Ave,37.869060,-122.293400,190
3,11th St at Bryant St,37.770030,-122.411726,828
4,11th St at Natoma St,37.773507,-122.416040,868


In [18]:
# Most popular station (station with the highest number of rides)
most_popular_station = station_data.loc[station_data['number_of_rides'].idxmax()]
print("Most popular station:", most_popular_station )


Most popular station: start_station_name         Market St at 10th St
start_station_latitude                37.776619
start_station_longitude             -122.417385
number_of_rides                            3904
Name: 199, dtype: object


In [19]:
# Least popular station (station with the lowest number of rides)
least_popular_station = station_data.loc[station_data['number_of_rides'].idxmin()]
print("Least popular station:", least_popular_station )

Least popular station: start_station_name         16th St Depot
start_station_latitude         37.766349
start_station_longitude      -122.396292
number_of_rides                        2
Name: 10, dtype: object


In [20]:
# Average number of rides per station
average_rides = station_data['number_of_rides'].mean()
print(f"Average number of rides per station: {average_rides:.2f}")

Average number of rides per station: 556.88


In [22]:
# Create a map centered around the city where the bike share system is located
city_center = [station_data['start_station_latitude'].mean(), station_data['start_station_longitude'].mean()]
map = folium.Map(location=city_center, zoom_start=12)

# Define marker colors based on the number of rides
def get_marker_color(rides):
    if rides < 100:
        return 'green'
    elif 100 <= rides < 500:
        return 'blue'
    else:
        return 'red'

# Plot each station on the map
for _, row in station_data.iterrows():
    folium.Marker(
        location=[row['start_station_latitude'], row['start_station_longitude']],
        popup=f"{row['start_station_name']}: {row['number_of_rides']} rides",
        icon=folium.Icon(color=get_marker_color(row['number_of_rides']))
    ).add_to(map)

# Display the map
map

In [23]:
# Save the map as an HTML file
map.save('bike_share_map.html')